In [23]:
import re
from random import uniform
from collections import defaultdict
import codecs

r_alphabet = re.compile(u'[а-яА-Я0-9-]+|[.,:;?!]+')

def gen_lines(corpus):
    data = codecs.open(corpus, 'r', "utf_8_sig")
    for line in data:
        yield line.lower()
        

def gen_tokens(lines):
    for line in lines:
        for token in r_alphabet.findall(line):
            yield token

def gen_trigrams(tokens):
    t0, t1 = '$', '$'
    for t2 in tokens:
        yield t0, t1, t2
        if t2 in '.!?':
            yield t1, t2, '$'
            yield t2, '$','$'
            t0, t1 = '$', '$'
        else:
            t0, t1 = t1, t2

def train(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)

    bi, tri = defaultdict(lambda: 0.0), defaultdict(lambda: 0.0)

    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1

    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]
    return model

def generate_sentence(model):
    phrase = ''
    t0, t1 = '$', '$'
    while 1:
        t0, t1 = t1, unirand(model[t0, t1])
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            phrase += ' ' + t1
    return phrase.capitalize()

def unirand(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return token


In [24]:
def generate_sentence_based(model, first_word='', n=100):
    phrase = ''
    if (len(first_word)>0):
        t0, t1 = '$', first_word
    else:
        t0, t1 = '$', '$'
    while n>0:
        n-=1
        t0, (t1, fr) = t1, unirand_freq(model[t0, t1])
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1 
        else:
            phrase += ' ' + t1
    return first_word.capitalize()+''+ phrase.capitalize()

In [25]:
def unirand_freq(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return (token, freq)

In [26]:
model = train('tolstoy.txt')

In [27]:
for i in range(10):
    print (generate_sentence(model))

Заметил ли граф безухов.
Сам по себе живет.
Однако перед экзаменом, когда все приезжавшие в укрепление, что будет с, с которой наташа прислушалась именно потому, что он сказал, что ты полезен!
В то, что с ним и не трогался с места, из настоящих.
-- да, это значило: что, ты мне пишешь, что бы ему отмстить, ты меня не удовлетворил ответ, что она еще спросила, -- сказал князь андрей; но он чувствовал по озлоблению стариков, по его спине.
Он тридцатилетним опытом знал, что государь знал о князе андрее, ни другого; а теперь он верно вам говорил, опустив голову и побить и забрать всех разом.
Он лежал и теперь пропитано водою.
Положим, что она не будет?
Если он будет таким же выражением досады проговорил денисов.
Слово теперь значило: при воронцове, то ли дело мы, отчего ж нам не сюда!


In [28]:
for i in range(10):
    print (generate_sentence_based(model, 'тогда'))

Тогда он был чиновником особых поручений.
Тогда как то делают историки, описывая поход ли 13-го года и восстановление бурбонов, и я оставлю детям десять тысяч в год кончины матушки авдотью васильевну?
Тогда жиран рвался так сильно занимали его.
Тогда бы я был под ульм, я мог бы никак объяснить; но он не богат, а туловище на другой берег и уже не состоял в том дело, стал показывать изорванные полы своей одежды, мне совершенно все равно, а хотелось по всем телодвижениям, было ли оно, это славно, -- сказал шиншин, с каким-то внешним добродушием и привычкою к нашему брату поступающему и нашему брату поступающему внушал зависть и уважение.
Тогда, придя в то мгновение, как и моих народов.
Тогда кому-нибудь из нашей армии; но ежели ты теперь занят?
Тогда отец назначил мне срок, дай я его очень ласково и покровительственно оглядывая танцующих.
Тогда я не знаю, отчего же вы, разлюбезные, -- говорила она матери.
Тогда исправить все человечество называет добром и даже те из членов, сколько бы все

In [29]:
import pandas as pd

In [30]:
df1 = pd.read_csv('24.csv')
df2 =pd.read_csv('lenta.csv')
df3 = pd.read_csv('vesti.csv')
df = pd.concat([df1,df2,df3])
del df1
del df2
del df3
df = df.dropna()

In [31]:
text = ' \n '.join(df.text)

In [36]:
file1 = codecs.open("MyFile.txt","w", "utf_8_sig") 
file1.write(text)
file1.close() 

In [38]:
model = train('MyFile.txt')
for i in range(10):
    print (generate_sentence(model))

В ноябре 2018 года в москве и санкт-петербурге.
29 5 127 видео рай и ад 1963 триллер, мистика, драма видео под покровом ночи 2016 криминал, мистика сша видео головокружение 1958 триллер, драма, мистика сша видео сердце ангела 1987 триллер, мелодрама, мистика, триллер сша видео малхолланд драйв, прирожд нные убийцы, экзистенция, молчание ягнят, женщина в песках 1963 драма, триллер сша видео сердце ангела 1987 триллер, драма, триллер сша видео прибытие поезда на вокзал ла-сьота 1895 документальный фильм колыма родина нашего страха, что у вышедшего осенью 2012-го диагональ экрана составляла 5, на корпусе р30 появился ответный разъем для 3, 7 мая.
2017.
54 5 49 видео подмена 2008 биографический фильм 0 23 маякрасивый, плохой, злойдрама, криминал франция, сша сыграли ключевую роль в восточной части европы происходили столкновения с ограждением.
Это означало, что она является представителем сми, ограничения, скорее всего, коснутся только новых моделей.
Мы должны помнить, что она носит то, чт

In [39]:
for i in range(10):
    print (generate_sentence(model))

Классический, литейный.
05 10: 15топ-10 худших аэропортов мира16.
Главный тренер сборной россии александра георгиева, который под давлением властей может отказаться от какой-либо финансовой компенсации и может отпустить футболиста, у коса нашла на камень при констриктивном перикардите, когда температура воздуха превышает отметку в 32 градуса.
Оба слота поддерживают сети четвертого 4 поколения, 12 человек госпитализировали, затем он набросился на начальницу поезда, идущего по телевизору постоянно показывают, что он их не существует метода для немедленного прекращения войны на стыке с верхним торцом, оказалась решетка разговорного динамика.
По делу о гибели с начала года увеличил добычу и снизил экспорт 16.
Я пишу об антигероях вместо того, кластер запустит партнерскую образовательную программу, посвященную финансовому и правовому сопровождению проектов.
Победу с большим удовольствием приеду как гость.
Важно, необходимо ежегодно выдавать 2, 1 млн рублей, на северо-западе и одним из немно

# Домашнее задание:

## Вариант 1: Чат бот в телеграм, который определяет тональность написанного в него текста

 - https://tproger.ru/translations/telegram-bot-create-and-deploy/
 - https://lmgtfy.com/?q=%D0%BE%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5+%D1%82%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8+%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0

## Вариант 2: Поиск именованных сущностей в новостях, маппинг их на географические названия страны, отметка по геопозиции

 - https://natasha.readthedocs.io/ru/latest/
 - https://proglib.io/p/python-web-map/